In [1]:
all = "∀"
exist = "∃"

In [2]:
import re

def eliminate_implication(term):
    match = re.search("->", term)
    if not match:
        return term
        
    start, end = match.span()
    term_chars = list(term)
    
    term_chars[start: end] = 'v'
    if term_chars[0] != '~':
        term_chars.insert(0, '~')     
    else:
        term_chars.pop(0)
        
    return "".join(term_chars)

print(eliminate_implication("X->Y"))
print(eliminate_implication("~X->Y"))
print(eliminate_implication("~X->~Y"))
print(eliminate_implication("X->~Y"))
print(eliminate_implication("~~X->Y"))

~XvY
XvY
Xv~Y
~Xv~Y
~XvY


In [3]:
def move_negation_inward(term):
    match1 = re.search(r"~\(", term)
    match2 = re.search("~~", term)
    match3 = re.search("~"+all, term)
    match4 = re.search("~"+exist, term)
    
    if not match1 and not match2 and not match3 and not match4:
        return term

    if match2:
        start, end = match2.span()
        return move_negation_inward(term[:start] + term[end:])
        
    if match1:
        start, end = match1.span()
        
        term_chars = list(term)
        term_chars.pop(start+1)
        term_chars.pop(-1)
        
        for i in range(len(term_chars)):
            if term_chars[i] == 'v':
                term_chars[i] = '^'
                term_chars.insert(i+1, '~')
                
            elif term_chars[i] == '^':
                term_chars[i] = 'v'
                term_chars.insert(i+1, '~')
                
        return move_negation_inward("".join(term_chars))

    if match3: 
        term_chars = list(term)
        if "".join(term_chars[:2]) == '~' + all:
            term_chars[:3] = exist + term_chars[term_chars.index(all)+1] +'~'
            return move_negation_inward("".join(term_chars))
            
    if match4:
        term_chars = list(term)
        
        if "".join(term_chars[:2]) == '~' + exist:
            term_chars[:3] = all + term_chars[term_chars.index(exist)+1] +'~'
            return move_negation_inward("".join(term_chars))


print(move_negation_inward("~(XvY)"))
print(move_negation_inward("~(X^Y)"))
print(move_negation_inward("~~(X^Y)")) # ~ ~(X^Y) => ~ ~Xv~Y => X^Y
print(move_negation_inward("~(~X^Y)")) # ~~Xv~Y => Xv~Y
print(move_negation_inward("~(~X^~Y)")) # ~~Xv~~Y => XvY
print(move_negation_inward("~~~X"))
print(move_negation_inward("~~X"))
print(move_negation_inward("~~~~X"))
print(move_negation_inward("~∀x~X"))
print(move_negation_inward("~∀xX"))
print(move_negation_inward("~∃xX"))
print(move_negation_inward("~∃x~X"))
print(move_negation_inward("~~∃x~X"))

~X^~Y
~Xv~Y
(X^Y)
Xv~Y
XvY
~X
X
X
∃xX
∃x~X
∀x~X
∀xX
∃x~X


In [4]:
# (∀x P( x)) v (∃x Q( x))
vars = ['X', 'Y', 'Z']
def standardize_variables(term):
    lst = term.split('v')
    for i in range(len(lst) - 1):
        lst[i+1] = ' '.join(map(lambda string: string.replace(vars[i], vars[i+1]), lst[i].split(' ')))
    return 'v'.join(lst)
    
standardize_variables("(∀X P(X)) v (∃X Q(X)) v (R(X))")

'(∀X P(X)) v(∀Y P(Y)) v(∀Z P(Z)) '

In [5]:
# (∀x P( x)) ∨ (∃y Q( y)) => ∀x ∃y P( x) ∨ Q( y)
lst = []

def move_all_quantifiers_left(term):
    if not all in term:
        
        return " ".join(lst) + term
        
    elif all in term:
        lst.append(all + term[term.index(all)+1])
        term = term.replace(all + term[term.index(all)+1], "")
        return move_all_quantifiers_left(term)

    
print(move_all_quantifiers_left("(∀x P( x)) ∨ (∀y Q( y))"))

∀x ∀y( P( x)) ∨ ( Q( y))


In [26]:
def skolemization(term):
    var = ""
    replacing_var = "A"
    if exist in term:
        var = term[term.index(exist) + 1]
        tmp = exist + var
        term = term.replace(tmp, "")
        if all in term:
            replacing_var = 'F(' + term[term.index(all) + 1] + ')'
        if replacing_var in(term):
            replacing_var = chr(ord(replacing_var) + 1)
        term = term.replace(var, replacing_var)
    
    return term

print(skolemization("∃y P( A) ∨ Q( y)"))
print(skolemization("∀x ∃y P( x) ∨ Q( y)"))

 P( A) ∨ Q( B)
∀x  P( x) ∨ Q( F(x))


In [38]:
# ∀x P( x) ∨ Q(F ( x)) → P( x) ∨ Q(F ( x))
def eliminate_universal_quantifiers(term):
    if all not in term:
        return term
    tmp = all + term[term.index(all) + 1]
    return eliminate_universal_quantifiers(term.replace(tmp, "").lstrip())

print(eliminate_universal_quantifiers("∀x ∀y ∀z P( x) ∨ Q(F ( x))"))
print(eliminate_universal_quantifiers("∀x ∀x ∀x P( x) ∨ Q(F ( x))"))
print(eliminate_universal_quantifiers("∀x P( x) ∨ Q(F ( x))"))

P( x) ∨ Q(F ( x))
P( x) ∨ Q(F ( x))
P( x) ∨ Q(F ( x))


In [67]:
# p ∨ (q ∧ r) → ( p ∨ q) ∧ ( p ∨ r)
def convert_to_CNF(term):
    lst1 = []
    lst2 = []
    str = ""
    for chr in term:
        if chr == '(' or chr == ')': pass
        elif chr == '^':
            lst2.append(str)
            str = str[:-3]
        else:
            str += chr
    lst2.append(str)
    print(lst2)
convert_to_CNF("p ^ (q v r)")
convert_to_CNF("p v (q ^ r)")

['p ', ' q v r']
['p v q ', 'p v r']
